In [1]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 42.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [2]:
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset

# Functions that produce the desired water level for stakeholders

In [ ]:
# 75.5  threshold for still water to flood along the South shoreline of Lake Ontario
# 74.4-75.45 preferred boating range

In [69]:
def house_owner(wl):
  return (75.5 - wl) * 10

In [46]:
def boating(wl):
  return 0

In [70]:
power_cap = (912.0 + 1045.0) * 10.0
def hydropower(wl, month):
  Q = 0.104
  g = 9.8
  return 0.9 * Q * g * wl

In [48]:
def navigation(wl):
  return 0

In [49]:
def envir(wl, month):
  return 0

# Calculating the optimal water level

In [50]:
def g(x):
  return x + 5

def p(c):
  return c * 2.0

In [75]:
# Constants in the constraints can be replaced by functions in the previous section

def Optwl():
    """Linear programming sample."""
    # Instantiate a Glop solver, naming it LinearExample.
    solver = pywraplp.Solver.CreateSolver("GLOP")
    if not solver:
        return

    # Water level
    wl = solver.NumVar(0, solver.infinity(), "water level")
    # y = solver.NumVar(0, solver.infinity(), "y")

    print("Number of variables =", solver.NumVariables())

    # Constraints of water level:
    solver.Add(wl >= 67.0)
    solver.Add(wl <= 86.0)

    # Constraints of house owners:
    solver.Add(house_owner(wl) >= 0.0)

    # Constraints of boaters:
    # solver.Add(boating(wl) <= 21.0)

    # Constraints of hydropower stations:
    solver.Add(hydropower(wl, 1) <= power_cap)
    # solver.Add(hydropower(wl, 1) >= 0.1 * power_cap)

    print("Number of constraints =", solver.NumConstraints())

    # Objective function:
    solver.Maximize(0.4 * house_owner(wl) + 0.3 * hydropower(wl, 1) + 0.2 * navigation(wl) + 0.1 * boating(wl))

    # Solve the system.
    print(f"Solving with {solver.SolverVersion()}")
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution:")
        print(f"Objective value = {solver.Objective().Value()}")
        print(f"Water level = {wl.solution_value()}m")
        #print(f"y = {y.solution_value()}")
    else:
        print("The problem does not have an optimal solution.")

    print("\nAdvanced usage:")
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")

In [76]:
Optwl()

Number of variables = 1
Number of constraints = 4
Solving with Glop solver v9.8.3296
Solution:
Objective value = 52.43732799999998
Water level = 67.0m

Advanced usage:
Problem solved in 5 milliseconds
Problem solved in 0 iterations
